In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
data=pd.read_csv(r'C:\Users\enriq\Apple\matches_pl.csv')
data.columns = [c.lower() for c in data.columns]

In [3]:
data['comp'].unique()

array(['Premier League', 'Europa Lg', 'EFL Cup', 'FA Cup',
       'Community Shield', 'Champions Lg', 'Conf Lg', 'Super Cup'],
      dtype=object)

In [4]:
#Only premier league matches are selected
data=data[data['comp']=='Premier League']

Variables importantes en defensive actions: 'tklw','tklw','int'

passgin'cmp','att','cmp%','totdist','prgdist','ast', 'xag', 'xa', 'kp', '1/3', 'ppa', 'crspa','prgp',

In [5]:
#A dichotomous variable is created where Loss or draw = 0, win=1. This variable will be the one we seek to predict
data["target"] = (data["result"] == "W").astype("int")

In [6]:
#dichotomous variable where local =1 and visitor=0.
data["venue_code"] = data["venue"].astype("category").cat.codes

In [7]:
#A variable is created where each opposing team is assigned a unique number.
data["opp_code"] = data["opponent"].astype("category").cat.codes

In [8]:
#Filter the columns
columns=['date', 'time', 'comp', 'round', 'day', 'venue', 'result',
       'gf', 'ga', 'opponent', 'xg_x', 'xga', 'poss','sh_x','sot%','g/sh', 'g/sot','dist','cmp','att_x','cmp%','totdist','prgdist','ast', 'xag', 'xa', 'kp', '1/3', 'ppa', 'crspa','prgp',
        'tklw','tkl%','int','clr','lost','season', 'team', 'target', 'venue_code', 'opp_code']
data=data[columns]

In [10]:
data.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg_x', 'xga', 'poss', 'sh_x', 'sot%', 'g/sh', 'g/sot',
       'dist', 'cmp', 'att_x', 'cmp%', 'totdist', 'prgdist', 'ast', 'xag',
       'xa', 'kp', '1/3', 'ppa', 'crspa', 'prgp', 'tklw', 'tkl%', 'int', 'clr',
       'lost', 'season', 'team', 'target', 'venue_code', 'opp_code'],
      dtype='object')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260 entries, 0 to 1609
Data columns (total 41 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1260 non-null   object 
 1   time        1260 non-null   object 
 2   comp        1260 non-null   object 
 3   round       1260 non-null   object 
 4   day         1260 non-null   object 
 5   venue       1260 non-null   object 
 6   result      1260 non-null   object 
 7   gf          1260 non-null   object 
 8   ga          1260 non-null   object 
 9   opponent    1260 non-null   object 
 10  xg_x        1260 non-null   float64
 11  xga         1260 non-null   float64
 12  poss        1260 non-null   float64
 13  sh_x        1260 non-null   float64
 14  sot%        1260 non-null   float64
 15  g/sh        1260 non-null   float64
 16  g/sot       1218 non-null   float64
 17  dist        1260 non-null   float64
 18  cmp         1260 non-null   float64
 19  att_x       1260 non-null  

In [12]:
df=data.copy()

In [13]:
#An important problem are the na values, so it is necessary to look at the columns with na values.
df.isna().sum()

date           0
time           0
comp           0
round          0
day            0
venue          0
result         0
gf             0
ga             0
opponent       0
xg_x           0
xga            0
poss           0
sh_x           0
sot%           0
g/sh           0
g/sot         42
dist           0
cmp            0
att_x          0
cmp%           0
totdist        0
prgdist        0
ast            0
xag            0
xa             0
kp             0
1/3            0
ppa            0
crspa          0
prgp           0
tklw           0
tkl%           0
int            0
clr            0
lost           0
season         0
team           0
target         0
venue_code     0
opp_code       0
dtype: int64

In [14]:
#Since there are few na, then these values ​​are changed by the mean of the variable
df["g/sot"] = df["g/sot"].fillna(df["g/sot"].mean())

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260 entries, 0 to 1609
Data columns (total 41 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1260 non-null   object 
 1   time        1260 non-null   object 
 2   comp        1260 non-null   object 
 3   round       1260 non-null   object 
 4   day         1260 non-null   object 
 5   venue       1260 non-null   object 
 6   result      1260 non-null   object 
 7   gf          1260 non-null   object 
 8   ga          1260 non-null   object 
 9   opponent    1260 non-null   object 
 10  xg_x        1260 non-null   float64
 11  xga         1260 non-null   float64
 12  poss        1260 non-null   float64
 13  sh_x        1260 non-null   float64
 14  sot%        1260 non-null   float64
 15  g/sh        1260 non-null   float64
 16  g/sot       1260 non-null   float64
 17  dist        1260 non-null   float64
 18  cmp         1260 non-null   float64
 19  att_x       1260 non-null  

In [16]:
df['team'].value_counts()

Arsenal                   64
TottenhamHotspur          64
Everton                   64
WolverhamptonWanderers    64
ManchesterCity            64
AstonVilla                63
Southampton               63
LeedsUnited               63
WestHamUnited             63
LeicesterCity             63
CrystalPalace             63
Chelsea                   63
Liverpool                 63
ManchesterUnited          63
NewcastleUnited           62
Brentford                 61
BrightonandHoveAlbion     61
Burnley                   38
Watford                   38
NorwichCity               38
NottinghamForest          25
Fulham                    25
Bournemouth               25
Name: team, dtype: int64

In [17]:
df['date']=pd.to_datetime(data['date'])

In [18]:
df.dtypes

date          datetime64[ns]
time                  object
comp                  object
round                 object
day                   object
venue                 object
result                object
gf                    object
ga                    object
opponent              object
xg_x                 float64
xga                  float64
poss                 float64
sh_x                 float64
sot%                 float64
g/sh                 float64
g/sot                float64
dist                 float64
cmp                  float64
att_x                float64
cmp%                 float64
totdist              float64
prgdist              float64
ast                  float64
xag                  float64
xa                   float64
kp                   float64
1/3                  float64
ppa                  float64
crspa                float64
prgp                 float64
tklw                 float64
tkl%                 float64
int                  float64
clr           

In [19]:
X = df.drop("target", axis=1)
y = df["target"]

In [20]:
#The data is divided into training and testing. But, because the time factor is important, neither cross validation nor 
#Bootstrap can be applied, so it is divided at one point in time.
model = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = df[df["date"] < '2022-11-01']
test = df[df["date"] > '2022-11-01']

In [21]:
test.shape

(248, 41)

In [22]:
predictors=["venue_code", "opp_code","sot%","g/sot","xg_x","xga","poss",'g/sh','dist','cmp','att_x','prgp','int','clr','lost']

In [23]:
df[predictors].isna().sum()

venue_code    0
opp_code      0
sot%          0
g/sot         0
xg_x          0
xga           0
poss          0
g/sh          0
dist          0
cmp           0
att_x         0
prgp          0
int           0
clr           0
lost          0
dtype: int64

columns=['date', 'time', 'comp', 'round', 'day', 'venue', 'result',
       'gf', 'ga', 'opponent', 'xg_x', 'xga', 'poss', 'attendance','sh_x','sot%','g/sh', 'g/sot','dist','cmp','att_x','cmp%','totdist','prgdist','ast', 'xag', 'xa', 'kp', '1/3', 'ppa', 'crspa','prgp',
        'tklw','int','clr','lost','season', 'team', 'target', 'venue_code', 'opp_code']

In [24]:
model.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [25]:
preds = model.predict(test[predictors])

In [26]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.8225806451612904

In [27]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
combined

,actual,predicted
18,1,1
20,1,1
21,1,1
22,1,1
23,0,0
...,...,...
650,0,0
651,0,0
652,1,1
653,0,0


In [28]:
combined.sum()

actual       98
predicted    90
dtype: int64

In [29]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,132,18
1,26,72


In [30]:
error=precision_score(test["target"], preds)
error

0.8

Aplicando medias moviles

In [31]:
grouped_matches = df.groupby("team")
group = grouped_matches.get_group("ManchesterCity").sort_values("date")

In [32]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,tklw,tkl%,int,clr,lost,season,team,target,venue_code,opp_code
656,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,11.0,26.3,3.0,11.0,14.0,2022,ManchesterCity,0,0,19
657,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,8.0,53.8,6.0,3.0,6.0,2022,ManchesterCity,1,1,16
658,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,10.0,75.0,4.0,3.0,2.0,2022,ManchesterCity,1,1,0
659,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,7.0,50.0,5.0,9.0,6.0,2022,ManchesterCity,1,0,11
661,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,10.0,27.8,8.0,13.0,13.0,2022,ManchesterCity,0,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2023-02-12,16:30,Premier League,Matchweek 23,Sun,Home,W,3.0,1.0,Aston Villa,...,3.0,30.8,3.0,3.0,9.0,2023,ManchesterCity,1,1,1
69,2023-02-15,19:30,Premier League,Matchweek 12,Wed,Away,W,3.0,1.0,Arsenal,...,12.0,58.3,9.0,14.0,5.0,2023,ManchesterCity,1,0,0
70,2023-02-18,15:00,Premier League,Matchweek 24,Sat,Away,D,1.0,1.0,Nott'ham Forest,...,6.0,25.0,2.0,5.0,3.0,2023,ManchesterCity,0,0,17
72,2023-02-25,17:30,Premier League,Matchweek 25,Sat,Away,W,4.0,1.0,Bournemouth,...,18.0,57.9,6.0,16.0,8.0,2023,ManchesterCity,1,0,2


In [33]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [34]:
cols=["sot%","g/sot","xg_x","xga","poss",'prgp','g/sh','dist','cmp','att_x','int','clr','lost']

new_cols = [f"{c}_rolling" for c in cols]
rolling_averages(group, cols, new_cols)


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,xga_rolling,poss_rolling,prgp_rolling,g/sh_rolling,dist_rolling,cmp_rolling,att_x_rolling,int_rolling,clr_rolling,lost_rolling
659,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,0.433333,70.666667,67.333333,0.150000,16.866667,641.000000,732.000000,4.333333,5.666667,7.333333
661,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,0.300000,69.333333,73.333333,0.163333,15.866667,655.000000,737.333333,5.000000,5.000000,4.666667
663,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,...,0.400000,68.333333,62.333333,0.080000,15.166667,592.333333,681.000000,5.666667,8.333333,7.000000
665,2021-10-03,16:30,Premier League,Matchweek 7,Sun,Away,D,2,2,Liverpool,...,0.400000,61.333333,56.000000,0.036667,15.933333,534.333333,625.000000,7.000000,13.000000,9.666667
666,2021-10-16,15:00,Premier League,Matchweek 8,Sat,Home,W,2,0,Burnley,...,0.500000,58.000000,43.000000,0.080000,16.833333,514.666667,612.666667,8.000000,16.000000,11.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2023-02-12,16:30,Premier League,Matchweek 23,Sun,Home,W,3.0,1.0,Aston Villa,...,1.000000,59.000000,51.333333,0.140000,16.266667,531.000000,618.666667,7.000000,14.333333,5.000000
69,2023-02-15,19:30,Premier League,Matchweek 12,Wed,Away,W,3.0,1.0,Arsenal,...,0.833333,62.000000,54.333333,0.100000,15.966667,545.000000,628.333333,5.333333,11.000000,6.666667
70,2023-02-18,15:00,Premier League,Matchweek 24,Sat,Away,D,1.0,1.0,Nott'ham Forest,...,1.100000,56.000000,42.000000,0.153333,15.200000,441.000000,529.666667,6.333333,9.666667,6.666667
72,2023-02-25,17:30,Premier League,Matchweek 25,Sat,Away,W,4.0,1.0,Bournemouth,...,0.966667,59.000000,52.333333,0.166667,15.266667,478.666667,572.666667,4.666667,7.333333,5.666667


In [35]:
matches_rolling = df.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,xga_rolling,poss_rolling,prgp_rolling,g/sh_rolling,dist_rolling,cmp_rolling,att_x_rolling,int_rolling,clr_rolling,lost_rolling
895,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1,0,Norwich City,...,2.900000,39.666667,32.000000,0.000000,14.833333,316.666667,402.333333,8.333333,19.000000,7.000000
896,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1,0,Burnley,...,2.733333,35.333333,28.333333,0.010000,14.133333,274.333333,356.333333,7.000000,20.000000,7.000000
898,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3,1,Tottenham,...,2.033333,41.666667,33.333333,0.036667,14.800000,326.000000,415.333333,8.666667,29.000000,5.666667
899,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0,0,Brighton,...,0.900000,50.333333,41.666667,0.120000,18.433333,404.666667,498.666667,8.000000,25.666667,6.666667
900,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2,2,Crystal Palace,...,1.033333,47.333333,35.666667,0.110000,19.833333,371.333333,469.333333,10.666667,26.333333,6.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,2023-02-11,15:00,Premier League,Matchweek 23,Sat,Away,W,2,1,Southampton,...,1.900000,48.000000,29.666667,0.076667,18.233333,414.000000,511.333333,6.333333,33.000000,5.333333
427,2023-02-18,15:00,Premier League,Matchweek 24,Sat,Home,L,0,1,Bournemouth,...,1.900000,48.333333,24.666667,0.086667,17.800000,391.666667,492.000000,7.000000,31.333333,8.333333
428,2023-02-24,20:00,Premier League,Matchweek 25,Fri,Away,D,1,1,Fulham,...,1.300000,55.666667,36.333333,0.086667,16.933333,432.333333,547.000000,4.000000,29.000000,9.333333
429,2023-03-01,20:00,Premier League,Matchweek 7,Wed,Away,L,0,2,Liverpool,...,0.866667,58.333333,38.000000,0.073333,16.833333,446.000000,563.000000,6.000000,21.333333,10.333333


In [36]:
matches_rolling.index = range(matches_rolling.shape[0])

In [37]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,xga_rolling,poss_rolling,prgp_rolling,g/sh_rolling,dist_rolling,cmp_rolling,att_x_rolling,int_rolling,clr_rolling,lost_rolling
0,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1,0,Norwich City,...,2.900000,39.666667,32.000000,0.000000,14.833333,316.666667,402.333333,8.333333,19.000000,7.000000
1,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1,0,Burnley,...,2.733333,35.333333,28.333333,0.010000,14.133333,274.333333,356.333333,7.000000,20.000000,7.000000
2,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3,1,Tottenham,...,2.033333,41.666667,33.333333,0.036667,14.800000,326.000000,415.333333,8.666667,29.000000,5.666667
3,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0,0,Brighton,...,0.900000,50.333333,41.666667,0.120000,18.433333,404.666667,498.666667,8.000000,25.666667,6.666667
4,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2,2,Crystal Palace,...,1.033333,47.333333,35.666667,0.110000,19.833333,371.333333,469.333333,10.666667,26.333333,6.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,2023-02-11,15:00,Premier League,Matchweek 23,Sat,Away,W,2,1,Southampton,...,1.900000,48.000000,29.666667,0.076667,18.233333,414.000000,511.333333,6.333333,33.000000,5.333333
1187,2023-02-18,15:00,Premier League,Matchweek 24,Sat,Home,L,0,1,Bournemouth,...,1.900000,48.333333,24.666667,0.086667,17.800000,391.666667,492.000000,7.000000,31.333333,8.333333
1188,2023-02-24,20:00,Premier League,Matchweek 25,Fri,Away,D,1,1,Fulham,...,1.300000,55.666667,36.333333,0.086667,16.933333,432.333333,547.000000,4.000000,29.000000,9.333333
1189,2023-03-01,20:00,Premier League,Matchweek 7,Wed,Away,L,0,2,Liverpool,...,0.866667,58.333333,38.000000,0.073333,16.833333,446.000000,563.000000,6.000000,21.333333,10.333333


In [48]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-11-01']
    test = data[data["date"] > '2022-11-01']
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [49]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [50]:
combined

,actual,predicted
47,1,1
48,1,1
49,1,1
50,1,1
51,0,0
...,...,...
1186,1,0
1187,0,0
1188,0,1
1189,0,0


In [51]:
error

0.8433734939759037

In [52]:
combined1, error1 = make_predictions(matches_rolling,new_cols)

In [53]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [54]:
combined.head(10)

,actual,predicted,date,team,opponent,result
47,1,1,2022-11-06,Arsenal,Chelsea,W
48,1,1,2022-11-12,Arsenal,Wolves,W
49,1,1,2022-12-26,Arsenal,West Ham,W
50,1,1,2022-12-31,Arsenal,Brighton,W
51,0,0,2023-01-03,Arsenal,Newcastle Utd,D
52,1,0,2023-01-15,Arsenal,Tottenham,W
53,1,1,2023-01-22,Arsenal,Manchester Utd,W
54,0,0,2023-02-04,Arsenal,Everton,L
55,0,0,2023-02-11,Arsenal,Brentford,D
56,0,0,2023-02-15,Arsenal,Manchester City,L


In [55]:
combined[combined['team']=='ManchesterCity']

,actual,predicted,date,team,opponent,result
724,1,1,2022-11-05,ManchesterCity,Fulham,W
725,0,0,2022-11-12,ManchesterCity,Brentford,L
726,1,1,2022-12-28,ManchesterCity,Leeds United,W
727,0,1,2022-12-31,ManchesterCity,Everton,D
728,1,1,2023-01-05,ManchesterCity,Chelsea,W
729,0,0,2023-01-14,ManchesterCity,Manchester Utd,L
730,1,1,2023-01-19,ManchesterCity,Tottenham,W
731,1,1,2023-01-22,ManchesterCity,Wolves,W
732,0,0,2023-02-05,ManchesterCity,Tottenham,L
733,1,1,2023-02-12,ManchesterCity,Aston Villa,W


In [56]:
combined[combined['team']=='Liverpool']

,actual,predicted,date,team,opponent,result
664,1,0,2022-11-06,Liverpool,Tottenham,W
665,1,1,2022-11-12,Liverpool,Southampton,W
666,1,1,2022-12-26,Liverpool,Aston Villa,W
667,1,0,2022-12-30,Liverpool,Leicester City,W
668,0,0,2023-01-02,Liverpool,Brentford,L
669,0,0,2023-01-14,Liverpool,Brighton,L
670,0,0,2023-01-21,Liverpool,Chelsea,D
671,0,0,2023-02-04,Liverpool,Wolves,L
672,1,1,2023-02-13,Liverpool,Everton,W
673,1,0,2023-02-18,Liverpool,Newcastle Utd,W


In [47]:
combined[combined['team']=='Chelsea']

,actual,predicted,date,team,opponent,result
345,0,0,2023-01-05,Chelsea,Manchester City,L
346,0,0,2023-01-12,Chelsea,Fulham,L
347,1,0,2023-01-15,Chelsea,Crystal Palace,W
348,0,0,2023-01-21,Chelsea,Liverpool,D
349,0,0,2023-02-03,Chelsea,Fulham,D
350,0,0,2023-02-11,Chelsea,West Ham,D
351,0,0,2023-02-18,Chelsea,Southampton,L
352,0,0,2023-02-26,Chelsea,Tottenham,L
353,1,0,2023-03-04,Chelsea,Leeds United,W
